In [12]:
pip install langchain-openai langchain-core ipywidgets python-docx PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import getpass
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from ipywidgets import FileUpload, Text, VBox, Output, Button, Layout
from docx import Document as DocxDocument
from PyPDF2 import PdfReader

In [2]:
# Set Azure OpenAI keys
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] = "62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "DskumarDeployment"
os.environ['OPENAI_TYPE'] = "Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [3]:
# Initialize the AzureChatOpenAI model
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.7,
    max_tokens=500,
    max_retries=2,
)

In [4]:
# Widgets for user interaction
upload_button = FileUpload(accept=".pdf,.docx", multiple=False)
brief_input = Text(placeholder="Enter a brief about the process you're automating...")
output_area = Output()
process_button = Button(description="Process", layout=Layout(width="150px"))

In [5]:
# Helper functions to extract content from uploaded files
def extract_text_from_docx(docx_file):
    doc = DocxDocument(docx_file)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    return "\n".join(page.extract_text() for page in reader.pages)

In [6]:
# Chat prompt template to guide the interaction
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a Technical Architect responsible for gathering requirements to provide 
               estimates for a software project. Ask focused questions to understand the 
               application's scope, tech stack, integration points, and complexity to break down tasks. 
               Make justified assumptions if details are missing and provide work estimates accordingly.""",
        ),
        ("human", "{input}"),
    ]
)

In [7]:
# Function to process file or brief input
def process_file_or_brief(_):
    output_area.clear_output()
    with output_area:
        if upload_button.value:
            uploaded_filename = list(upload_button.value.keys())[0]
            file_content = upload_button.value[uploaded_filename]["content"]

            if uploaded_filename.endswith(".pdf"):
                document = extract_text_from_pdf(file_content)
            elif uploaded_filename.endswith(".docx"):
                document = extract_text_from_docx(file_content)
            print(f"Extracted Content from {uploaded_filename}:\n", document)
            questions = generate_questions(document)
            print("\nGenerated Questions:\n", questions)
        else:
            brief = brief_input.value
            if brief:
                print("Brief provided by user:\n", brief)
                questions = generate_questions(brief)
                print("\nGenerated Questions:\n", questions)
            else:
                print("No document or brief provided. Please upload a file or enter a brief.")


In [8]:
# Function to generate questions using the Azure OpenAI model
def generate_questions(input_text):
    chain = prompt_template | llm
    response = chain.invoke({"input": input_text})
    return response.content

In [9]:
# Bind button click event
process_button.on_click(process_file_or_brief)

In [10]:
# Layout for the notebook interface
VBox([upload_button, brief_input, process_button, output_area])

In [11]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [12]:
# Set up Azure OpenAI credentials
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dskumar.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "DskumarDeployment"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"

In [13]:
# Initialize the AzureChatOpenAI model
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.7,
    max_tokens=1000,
    max_retries=2,
)

In [14]:
# Define the technical architect prompt
prompt_text = """
You are a Technical Architect. Your job is to arrive at the tech stack for a problem statement and provide estimates. 
You interact with the customer to ask questions one-by-one and gather necessary information.
Provide justifications for your estimates and list assumptions where required.
If the user doesn't provide required inputs, make reasonable assumptions to proceed.
"""

prompt = ChatPromptTemplate.from_messages([("system", prompt_text)])

In [15]:
# Function to extract content from the document
def read_document(path):
    if not path or not os.path.exists(path):
        return None

    if path.endswith('.txt'):
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    else:
        return "Unsupported file format. Only .txt files are supported for now."

In [16]:
# Main function to handle input
def process_document_or_questions(doc_path=None):
    content = read_document(doc_path)
    
    if content:
        print(f"Extracted Content from '{doc_path}':\n{content}\n")
    else:
        print("Invalid document path or no document provided. Starting Q&A to gather requirements.\n")
        chat_loop()

In [17]:
# Function to start the Q&A loop using Azure OpenAI
def chat_loop():
    conversation = [("system", prompt_text)]
    
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Exiting the conversation.")
            break

        # Add the user's input to the conversation
        conversation.append(("human", user_input))

        # Get the model's response
        response = llm.invoke(conversation)
        print(f"Assistant: {response.content}")

In [19]:
# Example Usage: Provide a document path or start Q&A
doc_path = input("Enter the document path (leave empty to skip): ").strip()
process_document_or_questions(doc_path)

Invalid document path or no document provided. Starting Q&A to gather requirements.

Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
Assistant: Hello! How can I assist you today?
Assistant: Sure, I can help you with that. Please provide me with the details of the problem statement or requirements that you have in mind.
Assistant: Can you please provide me with the problem statement?
Assistant: Thank you for reaching out. Could you please provide me with the details of the problem statement?
Assistant: Can you please provide me with the problem statement or requirements for the project?
Assistant: Sure, I'd be happy to help you out. Can you please provide me with the details of the problem statement?
Assistant: Thank you for reaching out. Can you please provide m

In [1]:
import os
import fitz  # PyMuPDF for PDF extraction
import docx  # For DOCX extraction
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Set up Azure OpenAI credentials
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dskumar.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "DskumarDeployment"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"

# Initialize the AzureChatOpenAI model
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.7,
    max_tokens=500,
    max_retries=2,
)

# Define the system prompt for the architect
system_prompt = """
You are a Technical Architect. Your goal is to gather requirements step-by-step by asking questions one-by-one.
Use each user response to frame follow-up questions until all necessary details are collected.
Provide justifications for your estimates and call out assumptions explicitly when needed.
"""

prompt = ChatPromptTemplate.from_messages([("system", system_prompt)])

# Function to extract content from TXT, PDF, or DOCX files
def extract_content(file_path):
    if file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()

    elif file_path.endswith(".pdf"):
        doc = fitz.open(file_path)
        content = ""
        for page in doc:
            content += page.get_text()
        doc.close()
        return content

    elif file_path.endswith(".docx"):
        doc = docx.Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])

    else:
        raise ValueError("Unsupported file format. Please upload TXT, PDF, or DOCX.")

# Main function to process the document or start Q&A
def process_document_or_questions(doc_path=None):
    try:
        if doc_path and os.path.exists(doc_path):
            content = extract_content(doc_path)
            print(f"Extracted Content from '{doc_path}':\n{content}\n")
        else:
            print("Invalid or missing document. Switching to Q&A mode.")
            start_interactive_qna()
    except Exception as e:
        print(f"Error: {e}")
        print("Starting interactive Q&A session.")
        start_interactive_qna()

# Q&A loop to interact with the user step-by-step
def start_interactive_qna():
    conversation = [("system", system_prompt)]

    while True:
        # Send the conversation to the model and get the response
        response = llm.invoke(conversation)
        print(f"Assistant: {response.content}")

        # Get user input or quit
        user_input = input("User: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting the conversation.")
            break

        # Add user's response to the conversation
        conversation.append(("human", user_input))


ModuleNotFoundError: No module named 'fitz'

In [ ]:
# Example Usage
doc_path = input("Enter the document path (leave empty to skip): ").strip()
process_document_or_questions(doc_path)

In [2]:
pip install langchain-openai langchain-core python-docx PyMuPDF fitz

  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
     ---- ----------------------------------- 1.0/9.5 MB 7.2 MB/s eta 0:00:02
     ---------------- ----------------------- 3.9/9.5 MB 11.2 MB/s eta 0:00:01
     ---------------------------------------  9.4/9.5 MB 14.3 MB/s eta 0:00:01
     ---------------------------------------- 9.5/9.5 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build 

  error: subprocess-exited-with-error
  
  × Building wheel for traits (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [283 lines of output]
      Reading version file C:\Users\817840\AppData\Local\Temp\pip-install-y1zilt2m\traits_25c479b697ba42eabae222b5c3079bcd\traits\version.py
      Package version from version file: ('6.3.2', '6df2ff9bd1d21c74e688aff6f67a19fbbefdd53b')
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\traits
      copying traits\api.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\base_trait_handler.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\constants.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\ctrait.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\editor_factories.py -> build\lib.win-amd64-cpython-311\traits
      copying traits\has_traits.py -> build\lib.win-amd64-cpython-311\traits
      copying trai